In [1]:
import math
import numpy as np
np.random.seed(578)
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
from tensorflow.python.framework import ops
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D,Conv1D,LSTM
from keras.layers.core import Reshape
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
K.set_image_dim_ordering('tf')

C:\Users\xiang\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [3]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
X_train = (X_train - X_train.mean(axis=0)) / (X_train.std(axis=0))
X_test = (X_test - X_test.mean(axis=0)) / (X_test.std(axis=0))
Y_train = kutils.to_categorical(Y_train)
Y_test = kutils.to_categorical(Y_test)

In [4]:
model = Sequential()
model.add(Conv2D(64, kernel_size = (5, 5), strides=(1, 1), input_shape=X_train.shape[1:], padding='same', activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size = (3, 3), strides=(1, 1), padding='same', activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size = (3, 3), strides=(1, 1), padding='same', activation="relu"))
model.add(Dropout(0.5))
model.add(Conv2D(256, kernel_size = (3, 3), strides=(1, 1), padding='same', activation="relu"))
model.add(Dropout(0.5))
model.add(Conv2D(256, kernel_size = (3, 3), strides=(1, 1), padding='same', activation="relu"))
model.add(Dropout(0.5))
model.add(Conv2D(128, kernel_size = (3, 3), strides=(1, 1), padding='same', activation="relu"))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [5]:
datagen = ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,
                            zca_whitening=True)

C:\Users\xiang\Anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py:514: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [6]:
datagen.fit(X_train)

In [ ]:
for i in range(len(X_test)):
    X_test[i] = datagen.standardize(X_test[i])

In [8]:
model.compile(loss="categorical_crossentropy", optimizer='Adam', metrics=["acc"])

In [9]:
class SaveHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses_train=[]
        self.acc_train=[]
        self.losses_test=[]
        self.acc_test=[]
    def on_epoch_end(self, epoch, logs={}):
        self.losses_train.append(logs.get('loss'))
        self.acc_train.append(logs.get('acc'))
        self.losses_test.append(logs.get('val_loss'))
        self.acc_test.append(logs.get('val_acc'))
history =SaveHistory()

In [10]:
batch_size = 100
epochs = 200

In [ ]:
model.fit_generator(datagen.flow(X_train, Y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, Y_test),validation_steps=X_test.shape[0] // batch_size, callbacks=[history])

In [ ]:
import pandas as pd
df = pd.DataFrame({'test_loss':history.losses_test, 
                   'train_loss':history.losses_train,
                   'test_acc': history.acc_test, 
                   'train_acc': history.acc_train 
                  })

In [ ]:
df.to_csv("alexnet_paper.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1,201),df['test_acc'])
plt.plot(range(1,201),df['train_acc'])

In [ ]:
import numpy as np
np.max(df['test_acc'])